In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import os

In [31]:
BASE_PATH = "/Users/harshit/Downloads/Research-Commons-Quant/automated-memetoken-index-pipeline/dataframes"
MONTHS = ["may23", "jun23", "jul23", "aug23"]  

In [32]:
monthly_token_data = {}

for month in MONTHS:
    files = glob.glob(f"{BASE_PATH}/{month}/*.csv")
    month_prices = {}
    month_volumes = {}

    for file in files:
        token = os.path.basename(file).replace(".csv", "")
        try:
            df = pd.read_csv(file)
            df.columns = df.columns.str.lower()
            if 'close' not in df.columns or 'volume' not in df.columns:
                continue

            df = df.sort_values(by='timestamp').reset_index(drop=True)
            df['days_since_launch'] = df.index
            month_prices[token] = df[['days_since_launch', 'close']].set_index('days_since_launch').rename(columns={'close': token})
            month_volumes[token] = df[['days_since_launch', 'volume']].set_index('days_since_launch').rename(columns={'volume': token})
        except Exception as e:
            print(f"⚠️ Error in {token}: {e}")

    if month_prices and month_volumes:
        aligned_prices = pd.concat(month_prices.values(), axis=1, join='inner')
        aligned_volumes = pd.concat(month_volumes.values(), axis=1, join='inner')

        monthly_token_data[month] = {
            "prices": aligned_prices,
            "volumes": aligned_volumes
        }

        print(f"✅ Loaded {month}: {aligned_prices.shape[1]} tokens, {aligned_prices.shape[0]} days")
    else:
        print(f"❌ Skipped {month}: No valid token data found")


✅ Loaded may23: 5 tokens, 229 days
✅ Loaded jun23: 5 tokens, 47 days
✅ Loaded jul23: 5 tokens, 377 days
✅ Loaded aug23: 5 tokens, 22 days
